In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API")

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", groq_api_key=groq_api_key)
model

c:\Users\pouru\Desktop\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002BF8F3D2050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002BF8F529C50>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="hi my name is pourush gupta, one of the greatest AI engineer")])


AIMessage(content='Hello, Pourush! It’s great to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says "hi my name is pourush gupta, one of the greatest AI engineer". Probably a greeting. Should respond politely, maybe ask how can I help. No disallowed content. So respond friendly.'}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 86, 'total_tokens': 159, 'completion_time': 0.156121441, 'completion_tokens_details': {'reasoning_tokens': 45}, 'prompt_time': 0.00323226, 'prompt_tokens_details': None, 'queue_time': 0.05277505, 'total_time': 0.159353701}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_df42644058', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--90883f1b-da2f-41dc-9566-dfd0a9b931a4-0', usage_metadata={'input_tokens': 86, 'output_tokens': 73, 'total_tokens': 159, 'output_token_details': {'reasoning': 45}})

In [5]:
from langchain_core.messages import HumanMessage, AIMessage

model.invoke(
    [
        HumanMessage(content="hi my name is pourush gupta, one of the greatest AI engineer"),
        AIMessage(content="Nice to meet you, Pourush Gupta, one of the greatest AI engineers. That's quite a bold statement, but I'm excited to learn more about your accomplishments and experiences in the field of Artificial Intelligence.\n\nAs a conversational AI, I'm designed to be a knowledge hub, and I'd love to engage in a conversation with you about the latest advancements in AI, your projects, or even your thoughts on the future of AI.\n\nWhat inspired you to become an AI engineer, and what specific areas of AI are you most passionate about?"),
        HumanMessage(content="What we were talking about?")
    ]

)

AIMessage(content='We’ve just started our conversation. Here’s a quick recap:\n\n1. **You introduced yourself** – you said your name is **Pourush Gupta** and described yourself as “one of the greatest AI engineers.”  \n2. **I responded** by greeting you, acknowledging the bold claim, and expressing interest in hearing about your work and achievements.  \n3. **I asked a follow‑up question** to keep the dialogue going:  \n   *What inspired you to become an AI engineer, and which areas of AI are you most passionate about?*  \n\nSo far we’ve just exchanged introductions and I’m waiting to hear more about your background, projects, and interests in AI. Let me know what you’d like to talk about next!', additional_kwargs={'reasoning_content': 'The user asks "What we were talking about?" They want a recap. The assistant should summarize the prior conversation: user introduced themselves as Pourush Gupta, one of the greatest AI engineer. Assistant responded with greeting and asked about inspira

### Message History

We can use a message history class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then laod those messages and pss them into the chain as part fo the input. 


In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
config = {"configurable":{"session_id":"chat1"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is pourush gupta, one of the greatest AI engineer")],
    config=config
)

In [11]:
response.content


'Hello Pourush! It’s great to meet you. How can I assist you today?'

In [20]:
config2 = {"configurable":{"session_id":"chat1"}}

response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config2
)

print(response.content)

Your name is Pourush Gupta.


In [14]:
print("\u202f")

### Prompt Templates

Prompt templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, whihc are we passing to the LLM.

In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [23]:
chain.invoke({"messages":[HumanMessage(content="hi my name is Pourush gupta and I am one of the greatest AI engineer")]})

AIMessage(content='Hello Pourush! It’s great to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'User says hi, introduces themselves. Likely wants a response. Should greet and respond politely. No disallowed content. So just greet back.'}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 107, 'total_tokens': 164, 'completion_time': 0.142845627, 'completion_tokens_details': {'reasoning_tokens': 30}, 'prompt_time': 0.004004607, 'prompt_tokens_details': None, 'queue_time': 0.054485613, 'total_time': 0.146850234}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_70d048ba3c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--1416d777-8400-4a3a-9dd7-04e9c906085d-0', usage_metadata={'input_tokens': 107, 'output_tokens': 57, 'total_tokens': 164, 'output_token_details': {'reasoning': 30}})